In [2]:
from langchain_openai import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate

# # Memory
# - 챗봇에 메모리를 추가하지 않으면 맥락을 기억할 수 없다.
# - 이전 대화 기록을 메모리에 저장하여 지속적인 대화가 가능하다.

# ### Conversation Buffer Memory
# - 단순히 이전 대화 내용 전체를 저장하는 메모리
# - 대화 내용이 길어질수록 메모리도 함께 커져 비효율적임
# - 모델에는 메모리가 없어 전체 대화 내용을 함께 보내야 모델이 이해할 수 있음.
# - 모든 메모리는 `save_context`, `load_memory_variables` 메소드를 갖고 있음

chat  = ChatOpenAI(model_name="gpt-4o-mini",
                temperature=0.1,
                streaming=True,
                callbacks=[StreamingStdOutCallbackHandler(),
                    ],
                )

# 1. 메시지 히스토리 인스턴스 생성
chat_history = ChatMessageHistory()

# 2. ConversationBufferMemory 구성 (chat_memory를 명시적으로 주입)
memory = ConversationBufferMemory(
    chat_memory=chat_history,
    return_messages=True  # True로 하면 AIMessage, HumanMessage 객체로 반환
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    ("human", "{input}"),
])

# 3. 컨텍스트 저장
memory.save_context({"input": "Hi I'm from S.Korea!"}, {"output": "It's Cool!"})

# 4. 컨텍스트 불러오기
memory_variables = memory.load_memory_variables({})
history = memory_variables["history"] # history라는 키 안에 대화 내용이 들어있음

user_input = "What did I say before?"

# 5. 과거 대화 + 새 질문을 합쳐 전체 메시지 시퀀스를 구성
messages = history + prompt.format_messages(input=user_input)
# 모델 호출 & 응답 출력
response = chat.invoke(messages)

# 6. 질문(input)과 응답(output)을 메모리에 저장
memory.save_context({"input": user_input}, {"output": response.content})

# 7. 메모리에서 대화 기록 불러오기(Dev_확인용)
last_history = memory.load_memory_variables({})["history"]
for message in last_history:
    print(f"{message.type}: {message.content}")


You mentioned that you are from South Korea. How can I assist you today?human: Hi I'm from S.Korea!
ai: It's Cool!
human: What did I say before?
ai: You mentioned that you are from South Korea. How can I assist you today?
